In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [ ]:
prediction = pd.DataFrame()
for i in range(0, len(dfB)-3):
    df = dfB[0:i+3]
    df = pd.DataFrame(df)
    scaler=MinMaxScaler(feature_range=(0,1))
    scaled_data=scaler.fit_transform(df.values.reshape(-1,1))
    prediction_days=1
    x_train=[]
    y_train=[]
    
    for x in range(prediction_days, len(scaled_data)):
        x_train.append(scaled_data[x-prediction_days:x,0])
        y_train.append(scaled_data[x,0])
    
    x_train, y_train=np.array(x_train),np.array(y_train)
    x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
    model=Sequential()
    model.add(LSTM(units=50,return_sequences=True, input_shape=(x_train.shape[1],1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(optimizer='adam',loss='mean_squared_error')
    model.fit(x_train,y_train,epochs=25,batch_size=32)
    test_data = df[len(df)-prediction_days:len(df)]
    model_inputs=df[len(df["Value"])-len(test_data)-prediction_days:].values
    model_inputs=model_inputs.reshape(-1,1)
    model_inputs=scaler.transform(model_inputs)

    x_test=[]
    for x in range(prediction_days,len(model_inputs)):
        x_test.append(model_inputs[x-prediction_days:x,0])
        x_test=np.array(x_test)
    x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
    predicted_prices=model.predict(x_test)
    predicted_prices=scaler.inverse_transform(predicted_prices)
    predictions = pd.DataFrame(predicted_prices)
    predictions= pd.DataFrame(predictions)
    prediction = prediction.append(predictions)
